In [1]:
import time
import pybullet as p
import pybullet_data
import math
import numpy as np

frequence = 120
timeStep = 1/frequence

baseAngle = np.pi / 6

velocityGain = 0.5
force = 1000
positionGain = 0.3

# Connect to PyBullet and set up the environment
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81)
p.setRealTimeSimulation(0)  # Use step simulation for better control
p.setTimeStep(timeStep)  # Set a higher time step for smoother simulation


camera_target_position = [0, 0, 0] # Position to look at
camera_distance = 1 # Distance from the target
camera_yaw = 165 # Yaw angle
camera_pitch = -40 # Pitch angle

# Calculate the camera position
camera_position = [
camera_target_position[0] + camera_distance * math.sin(math.radians(camera_yaw)) * math.cos(math.radians(camera_pitch)),
camera_target_position[1] + camera_distance * math.sin(math.radians(camera_pitch)),
camera_target_position[2] + camera_distance * math.cos(math.radians(camera_yaw)) * math.cos(math.radians(camera_pitch))
]
# Set the camera
p.resetDebugVisualizerCamera(cameraDistance=camera_distance,
cameraYaw=camera_yaw,
cameraPitch=camera_pitch,
cameraTargetPosition=camera_target_position)


# Load the plane and the mill URDF model
plane_id = p.loadURDF("plane.urdf")
robot_id = p.loadURDF("hexapod_2.urdf", useFixedBase=0, basePosition=[0, 0, 0.14 - np.sin(baseAngle)*0.08], baseOrientation=[0, 0, 0, 1],)



# Joints info
jointIndex = {}
revoluteJointIndex = {}
endEffectorIndex = {}
numJoints = p.getNumJoints(robot_id)
for i in range(numJoints):
    jointInfo = p.getJointInfo(robot_id, i)
    key = jointInfo[1].decode('utf-8')
    jointIndex[key] = i
    if key[0] == 'u' or key[0] == 'v' or key[0] == 'w':
        revoluteJointIndex[key] = i
    elif key[0] == 'x':
        endEffectorIndex[key] = i
N = len(revoluteJointIndex)
print(revoluteJointIndex)

# reset joint pos
for name, idx in jointIndex.items():
    if name[0] == 'v': 
        p.resetJointState(robot_id, idx, baseAngle)
    elif name[0] == 'w' :
        p.resetJointState(robot_id, idx, -baseAngle*4)
        

# Set join freewheel
p.setJointMotorControlArray(
    bodyIndex=robot_id,
    jointIndices=revoluteJointIndex.values(),
    controlMode=p.VELOCITY_CONTROL,
    targetVelocities=[0]*N,
    forces=[0]*N  # No force applied, joint should rotate freely
)

# Optionally, you can disable joint damping and friction directly if needed
for n in revoluteJointIndex.values():
    p.changeDynamics(
        robot_id,
        n,
        linearDamping=0,
        angularDamping=0, # doesnt change anything
        jointDamping=0 # >0 Not working
    )






# Run the simulation
time.sleep(0.5)
for t in range(frequence * 100):  


    
    p.stepSimulation()
    time.sleep(timeStep)  # Sleep to maintain the desired time step

# Disconnect from the simulation

time.sleep(0.5)
p.disconnect()


pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
{'u1': 1, 'v1': 2, 'w1': 3, 'u2': 6, 'v2': 7, 'w2': 8, 'u3': 11, 'v3': 12, 'w3': 13, 'u4': 16, 'v4': 17, 'w4': 18, 'u5': 21, 'v5': 22, 'w5': 23, 'u6': 26, 'v6': 27, 'w6': 28}
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) UHD Graphics 620 (KBL GT2)
GL_VERSION=4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Intel
Renderer = Mesa Intel(R) UHD Graphics 620 (KBL GT2)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFun

: 

In [10]:

# _60 = np.pi / 3

# leg_length = 0.08 + 0.05 + 0.05 + np.cos(baseAngle)*0.08
# leg_height = np.sin(baseAngle)* 0.08 - 0.14
# l = np.cos(_60)*leg_length
# L = np.sin(_60)*leg_length

# rest_positions = {
#     'leg_1': [0, leg_length, leg_height],
#     'leg_2': [L, l, leg_height],
#     'leg_3': [L, -l, leg_height],
#     'leg_4': [0, -leg_length, leg_height],
#     'leg_5': [-L, -l, leg_height],
#     'leg_6': [-L, l, leg_height]
# }


base_position = []

for key, idx in endEffectorIndex.items():
    linkState = p.getLinkState(robot_id, idx)
    base_position.append(list(linkState[0]))


print(base_position)

for i in range(len(base_position)):
    for j in range(3):
        base_position[i][j] += np.random.uniform(-0.1,0.1)



#positions = p.calculateInverseKinematics(robot_id,jointIndex['x1'], [x,y,z])
positions = p.calculateInverseKinematics2(robot_id, endEffectorIndex.values(), base_position)

# p.setJointMotorControl2(robot_id, jointIndex['u1'], p.POSITION_CONTROL, positions[0])
# p.setJointMotorControl2(robot_id, jointIndex['v1'], p.POSITION_CONTROL, positions[1])
# p.setJointMotorControl2(robot_id, jointIndex['w1'], p.POSITION_CONTROL, positions[2])
p.setJointMotorControlArray(robot_id, revoluteJointIndex.values(), p.POSITION_CONTROL, positions)
    
for t in range(frequence * 1):

    p.stepSimulation()
    time.sleep(timeStep)

[[0.0226897919510321, 0.18738729991096456, 0.1589462172468317], [0.21294746828695765, 0.1749150147786624, 0.21087018302375182], [0.20922192642909954, -0.11814803283297408, 0.18300014555188943], [-0.011795475176628762, -0.1922728170227861, 0.1868530162312277], [-0.19166743081072338, -0.18655513716442237, 0.21010388809500474], [-0.1907785779357033, 0.24287992403575692, 0.24870030120349707]]


In [4]:
p.disconnect()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
